### Assignment 2
Explain how the program is compiled and run.
For a vector length of N:

1. How many floating operations are being performed in your vector add kernel?
2. How many global memory reads are being performed by your kernel?

For a vector length of 512:

1. Explain how many CUDA threads and thread blocks you used.
2. Profile your program with Nvidia Nsight. What Achieved Occupancy did you get? You might find https://docs.nvidia.com/nsight-compute/NsightComputeCli/index.html#nvprof-metric-comparison
Links to an external site. useful.

Now increase the vector length to 262140:

1. Did your program still work? If not, what changes did you make?
2. Explain how many CUDA threads and thread blocks you used.
Profile your program with Nvidia Nsight. What Achieved Occupancy do you get now?

Further increase the vector length (try 6-10 different vector length), plot a stacked bar chart showing the breakdown of time including (1) data copy from host to device (2) the CUDA kernel (3) data copy from device to host. For this, you will need to add simple CPU timers to your code regions.





In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Wed Nov 13 13:38:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              13W /  7


## Next, we write a native CUDA code and save it as 'vectorAdd.cu'


In [2]:
%%writefile vectorAdd.cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values
c = 0;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

Writing vectorAdd.cu


In [33]:
%%writefile hw2_ex1.cu
#include <stdio.h>
#include <sys/time.h>

#define DataType double

__global__ void vecAdd(DataType *in1, DataType *in2, DataType *out, int len) {
  //@@ Insert code to implement vector addition here
  int myID = blockIdx.x*blockDim.x + threadIdx.x;
  if (myID < len) {
    out[myID] = in1[myID] + in2[myID];
  }
}

//@@ Insert code to implement timer start

//@@ Insert code to implement timer stop


int main(int argc, char **argv) {
  int inputLength;
  DataType *hostInput1;
  DataType *hostInput2;
  DataType *hostOutput;
  DataType *resultRef;
  DataType *deviceInput1;
  DataType *deviceInput2;
  DataType *deviceOutput;

  //@@ Insert code below to read in inputLength from args
  if (argc < 2) {
    printf("Please provide a number as a command-line argument.\n");
    return 1;
  }
  char *end;
  inputLength = strtol(argv[1], &end, 10);  // base 10 for decimal
  if (*end != '\0') {
    printf("Invalid integer: %s\n", argv[1]);
    return 1;
  }

  printf("The input length is %d\n", inputLength);

  //@@ Insert code below to allocate Host memory for input and output
  int size = inputLength * sizeof(DataType);
  hostInput1 = (double *)malloc(size);
  hostInput2 = (double *)malloc(size);
  hostOutput = (double *)malloc(size);
  resultRef = (double *)malloc(size);

  //@@ Insert code below to initialize hostInput1 and hostInput2 to random numbers, and create reference result in CPU
  srand(time(NULL)); // use current time as seed for random generator
  for(int i =0; i < inputLength; ++i) {
    hostInput1[i] = static_cast<DataType>(rand())/RAND_MAX;
    hostInput2[i] = static_cast<DataType>(rand())/RAND_MAX;
  }

  //@@ Insert code below to allocate GPU memory here
  cudaMalloc(&deviceInput1, size);
  cudaMalloc(&deviceInput2, size);
  cudaMalloc(&deviceOutput, size);

  //@@ Insert code to below to Copy memory to the GPU here
  cudaMemcpy(deviceInput1, hostInput1, size, cudaMemcpyHostToDevice);
  cudaMemcpy(deviceInput2, hostInput2, size, cudaMemcpyHostToDevice);


  //@@ Initialize the 1D grid and block dimensions here
  dim3 numThreadsPerBlock = 256;
  dim3 numBlocks;
  numBlocks.x = (inputLength + numThreadsPerBlock.x - 1)/numThreadsPerBlock.x;

  // Create CUDA events
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // Record the start event
  cudaEventRecord(start, 0);

  // Launch the kernel
  vecAdd<<<gridSize, blockSize>>>(deviceInput1, deviceInput2, deviceOutput, inputLength);

  // Record the stop event
  cudaEventRecord(stop, 0);

  // Wait for the stop event to complete
  cudaEventSynchronize(stop);

  // Calculate the elapsed time
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);

  printf("Kernel execution time: %.4f ms\n", milliseconds);

  cudaError_t kernel_err = cudaGetLastError();
  if (kernel_err != cudaSuccess) {
    printf("Kernel launch error: %s\n", cudaGetErrorString(kernel_err));
  }

  //@@ Copy the GPU memory back to the CPU here
  cudaError err = cudaMemcpy(hostOutput, deviceOutput, size, cudaMemcpyDeviceToHost);
  if(err != cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }

  //@@ Insert code below to compare the output with the reference
  for(int i = 0; i < inputLength; ++i) {
    resultRef[i] = hostInput1[i] + hostInput2[i];
  }
  for(int i = 0; i < inputLength; ++i) {
    if (abs(resultRef[i] - hostOutput[i]) > 1e-5) {
      printf("Addition wrong at %i: (cpu, gpu) = (%.3f, %.3f)", i, resultRef[i], hostOutput[i]);
      break;
    }
  }

  //@@ Free the GPU memory here
  cudaFree(deviceInput1);
  cudaFree(deviceInput2);
  cudaFree(deviceOutput);

  //@@ Free the CPU memory here
  free(hostInput1);
  free(hostInput2);
  free(hostOutput);
  free(resultRef);

  return 0;
}

Overwriting hw2_ex1.cu


## We compile the saved cuda code using nvcc compiler

In [34]:
!nvcc hw2_ex1.cu -o hw2_ex1
!ls


hw2_ex1  hw2_ex1.cu  sample_data  vectorAdd.cu


## Finally, we execute the binary of the compiled code

In [35]:
!./hw2_ex1 10

The input length is 10


In [7]:
# !./vectorAdd
!nvprof --print-gpu-trace ./vectorAdd

==2715== NVPROF is profiling process 2715, command: ./vectorAdd
result is 8
==2715== Profiling application: ./vectorAdd
==2715== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
168.96ms  1.1840us                    -               -         -         -         -        4B  3.2219MB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
168.97ms     672ns                    -               -         -         -         -        4B  5.6766MB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
169.18ms  3.3600us              (1 1 1)         (1 1 1)        16        0B        0B         -           -           -           -     Tesla T4 (0)         1         7  add(int*, int*, int*) [130]
169.19ms  2.0800us                    -               -         -         -         -        4B  1.834

In [9]:
!ncu ./vectorAdd

==PROF== Connected to process 3292 (/content/vectorAdd)
==PROF== Profiling "add(int *, int *, int *)" - 0: 0%....50%....100% - 8 passes
result is 8
==PROF== Disconnected from process 3292
[3292] vectorAdd@127.0.0.1
  add(int *, int *, int *) (1, 1, 1)x(1, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         4.63
    SM Frequency            cycle/usecond       546.22
    Elapsed Cycles                  cycle        1,993
    Memory Throughput                   %         0.54
    DRAM Throughput                     %         0.08
    Duration                      usecond         3.65
    L1/TEX Cache Throughput             %        19.98
    L2 Cache Throughput                 %         0.54
    SM Active Cycles                cycle        20